In [3]:
import sys
import re
import pandas as pd
import json
import os
import dotenv

from pathlib import Path
from typing import overload
from langchain_core.documents import Document
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.chains import ConversationalRetrievalChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain.prompts import PromptTemplate
from langchain_cohere import CohereEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from sentence_transformers import SentenceTransformer
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.vectorstores import VectorStore
from openai import OpenAI

dotenv.load_dotenv()

c:\Users\Korn\Desktop\dsi314\Chatbot-Tourism-Recommendation-for-Pathum-Thani-Using-RAG\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [4]:
os.chdir("C:\\Users\\Korn\\Desktop\\dsi314\\Chatbot-Tourism-Recommendation-for-Pathum-Thani-Using-RAG")
print(os.getcwd())

C:\Users\Korn\Desktop\dsi314\Chatbot-Tourism-Recommendation-for-Pathum-Thani-Using-RAG


In [5]:
def restructure_json(input_path: str, main_key: str = "name"):
    with open(input_path, "r", encoding="utf-8") as file:
        data = json.load(file)
    
    updated_data = {main_key: data}
    
    with open(input_path, "w", encoding="utf-8") as file:
        json.dump(updated_data, file, ensure_ascii=False, indent=4)

@overload
def extract_data(path: str, main_key: str)->list[Document]:...

@overload
def extract_data(path: str)->list[Document]:...

def extract_data(path: str, main_key:str="name")->list[Document]:
    documents: list[Document] = []
    with open(path, "r", encoding='utf8') as json_file:
        data:list[dict[str, str]] = json.load(json_file)[main_key]

        for each_data in data:
          page_content: str = ""
          for k in each_data:
              page_content = page_content + f"{k} {each_data[k]}"
          current_document = Document(
              page_content=page_content,
          )
          documents.append(current_document)
    return documents

In [6]:
# Load docs
doc_paths = [
    "./data\darft.pdf",
    "./data\draft2.pdf",
    "./data\สามโคก ร้านอาหาร คาเฟ่ เมือง คา.csv",
]

docs = []
for doc_file in doc_paths:
    file_path = Path(doc_file)
    file_name = os.path.splitext(os.path.basename(file_path))[0]

    try:
        if str(file_path).endswith(".pdf"):
            loader = PyMuPDFLoader(file_path=file_path)
            data = loader.load_and_split()
        elif str(file_path).endswith(".csv"):
            df = pd.read_csv(file_path)
            df = df[['name','description','reviews','website','phone','main_category','rating','workday_timing','closed_on','address','link']]
            df.to_json(f'./data\cleaned_{file_name}.json', force_ascii=False, orient='records')            
            restructure_json(f'./data\cleaned_{file_name}.json')
            data = extract_data(f'./data\cleaned_{file_name}.json')
        else:
            print('Upload a file successful')
            sys.exit()

        docs.extend(data)

    except Exception as e:
        print(f"Error loading document {file_name}: {e}")


In [7]:
# Split docs
text_splitter = CharacterTextSplitter(separator="\n\n", chunk_size=512, chunk_overlap=0)
texts = text_splitter.split_documents(docs)

In [8]:
for i in range(len(texts)):
    cleaned_content = re.sub(r'[\u202a-\u202e]', '', texts[i].page_content)
    texts[i] = Document(metadata=texts[i].metadata, page_content=cleaned_content)

print(texts[140])

page_content='○ วัดบางหลวงสร้างในสมัยกรุงศรีอยุธยา อุโบสถหลังเก่าเป็นที่ประดิษฐาน หลวงพ่อใหญ่ ซึ่งเป็น
พระประธานที่มีขนาดใหญ่ที่สุดของเมืองปทุมธานี ส่วนหลวงพ่อเพชร เป็นพระพุทธรูปสมัย
เชียงแสน มีพุทธลักษณะพิเศษ คือ พระหัตถ์ซ้ายบิดไปมาได้ ทางวัดจะอัญเชิญหลวงพ่อเพชรออก
มาให้ประชาชนได้สักการะในช่วงเทศกาลออกพรรษา นอกจากนี้ วัดบางหลวงมีการกวนข้าวทิพย์
ในวันที่ 10-11 เมษายนของทุกปี
○ วัดหงส์ปทุมาวาสหรือ วัดมอญ เป็นวัดราษฎร์สังกัดคณะสงฆ์ฝ่ายมหานิกาย ตั้งอยู่ริมแม่น้ำ
เจ้าพระยา ในตำบลบางปรอก อำเภอเมืองปทุมธานี จังหวัดปทุมธานี เป็นวัดได้รับรางวัลชนะเลิศ
โครงการอนุรักษ์พันธุ์ปลาหน้าวัดด้วยความที่มีพันธุ์ปลาต่าง ๆ มากมาย ไม่ว่าจะเป็นปลาสวาย
ปลาเทโพ ปลาแรด ปลาตะเพียน และปลาอื่นอีกหลายชนิด มีที่ดินที่ตั้งวัดเนื้อที่ 12 ไร่ 1 งาน 52
ตารางวา โดยวัดหงส์ปทุมาวาสสร้างขึ้นเมื่อ พ.ศ. 2317 โดยชาวมอญที่อพยพหนีพม่ามาไทยใน
สมัยพระเจ้ากรุงธนบุรี เดิมมีชื่อว่า วัดหงษา จนกระทั่งใน พ.ศ. 2485 จึงเปลี่ยนชื่อมาเป็น "วัด
หงษ์ปทุมาวาส" ได้รับพระราชทานวิสุงคามสีมาเมื่อวันที่ 30 ธันวาคม พ.ศ. 2524 เขต
วิสุงคามสีมากว้าง 40

In [9]:
#HuggingFaceEmbeddings
model_name = "BAAI/bge-m3"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

c:\Users\Korn\Desktop\dsi314\Chatbot-Tourism-Recommendation-for-Pathum-Thani-Using-RAG\venv\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Korn\.cache\huggingface\hub\models--BAAI--bge-m3. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [10]:
# storing embeddings in the vector store
vectorstore = FAISS.from_documents(texts, embeddings)

In [13]:
save_path = "./vectorstore_directory"
vectorstore.save_local(save_path)
print(f"Vector Store saved to {save_path}")

Vector Store saved to ./vectorstore_directory


In [14]:
load_path = "./vectorstore_directory"
vectorstore = FAISS.load_local(load_path, embeddings, allow_dangerous_deserialization=True)
print("Vector Store loaded successfully!")

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

Vector Store loaded successfully!


In [15]:
typhoon_prompt = PromptTemplate(
    input_variables=["context","question"],
    template="""
    You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question.
    You should answer the question in Thai language only.
    Context: {context}

    You are an expert travel guide specializing in tourist attractions in Pathum Thani, Thailand.
    The user has the following question:
    Question: {question}

    Please provide a helpful response with the following details:
    1. Name of the attraction
    2. Description of the place (e.g., unique features, activities available)
    3. Opening and closing hours
    4. Additional information (e.g., transportation tips, entrance fees, or special advice)
    
    If you don't know the answer, simply say, "I don't know."
    """,
)

In [20]:
#typhoon_token = os.getenv("TYPHOON_API_KEY")

# Initialize the Typhoon client
client = OpenAI(
    api_key=os.getenv("TYPHOON_API_KEY"),
    base_url='https://api.opentyphoon.ai/v1'
)

# Define a function to generate a response using the LLM
def generate_response(context, chat_history, question):
    history = "\n".join([f"User: {entry['user']}\nAssistant: {entry['assistant']}" for entry in chat_history])
    prompt = f"{typhoon_prompt}\n\n{history}\n\nContext: {context}\n\nUser: {question}\nAssistant:"
    chat_completion = client.chat.completions.create(
        model="typhoon-v1.5x-70b-instruct",
        messages=[{"role": "user", "content": prompt}],
    )
    return chat_completion.choices[0].message.content

# Main function to handle user query
def answer_question(user_question, chat_history):
    retrieved_contexts = retriever.get_relevant_documents(user_question)
    context = "\n".join([doc.page_content for doc in retrieved_contexts])
    response = generate_response(context=context, chat_history=chat_history, question=user_question)
    return response

In [21]:
# Main loop with chat history
chat_history = []
query = None

while True:
    if not query:
        query = input("Please ask your question (or type 'quit' to exit): ")
    if query.lower() in ['quit', 'exit', 'q']:
        print("Exiting chat. Goodbye!")
        break
    answer = answer_question(query, chat_history)
    chat_history.append({"user": query, "assistant": answer})
    print("\nAssistant:", answer)
    query = None


Assistant: สวัสดีค่ะ! ฉันเป็นผู้ช่วยในการตอบคำถามเกี่ยวกับการท่องเที่ยวในจังหวัดปทุมธานี, ประเทศไทย. คุณมีคำถามอะไรเกี่ยวกับสถานที่ท่องเที่ยวหรือไม่?

Assistant: สวัสดีค่ะ! ฉันเป็นผู้ช่วยในการตอบคำถามเกี่ยวกับการท่องเที่ยวในจังหวัดปทุมธานี, ประเทศไทย. คุณมีคำถามอะไรเกี่ยวกับสถานที่ท่องเที่ยวหรือไม่?

Assistant: Hello! I am an assistant for question-answering tasks specializing in tourist attractions in Pathum Thani, Thailand. Do you have any questions about tourist attractions?

Context:
1. ร้านอาหารริมน้ำผู้ใหญ่สุวิทย์
Description: None
Reviews: 299
Website: https://www.wongnai.com/restaurants/80037cs-%E0%B8%A3%E0%B9%89%E0%B8%B2%E0%B8%99%E0%B8%AD%E0%B8%B2%E0%B8%AB%E0%B8%B2%E0%B8%A3%E0%B8%9C%E0%B8%B9%E0%B9%89%E0%B9%83%E0%B8%AB%E0%B8%8D%E0%B9%88%E0%B8%AA%E0%B8%B8%E0%B8%A7%E0%B8%B4%E0%B8%97%E0%B8%A2%E0%B9%8C
Phone: 088 332 9559
Main Category: Restaurant
Rating: 3.9
Workday Timing: 10:00–20:00
Closed On: Open All Days
Address: 55 1 ตำบลบ้านปทุม อำเภอสามโคก ปทุมธานี 12160
Link: https://ma